In [1]:
import tensorflow as tf
import time
import os
import numpy as np

In [3]:
# Convolution Block

def _conv(x,kernel,name,log=False):
    with tf.variable_scope(name):
        W = tf.get_variable(initializer=tf.truncated_normal(shape=kernel,stddev=0.01),name='W')
        b = tf.get_variable(initializer=tf.constant(0.0,shape=[kernel[3]]),name='b')
        conv = tf.nn.conv2d(x, W, strides=[1,1,1,1],padding='SAME')
        activation = tf.nn.relu(tf.add(conv,b))
        pool = tf.nn.max_pool(activation,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
        if log==True:
            tf.summary.histogram("weights",W)
            tf.summary.histogram("biases",b)
            tf.summary.histogram("activations",activation)
        return pool

# Dense Block

def _dense(x,size_in,size_out,name,relu=False,log=False):
    with tf.variable_scope(name):
        flat = tf.reshape(x,[-1,size_in])
        W = tf.get_variable(initializer=tf.truncated_normal([size_in,size_out],stddev=0.1),name='W')
        b = tf.get_variable(initializer=tf.constant(0.0,shape=[size_out]),name='b')
        activation = tf.add(tf.matmul(flat,W),b)
        if relu==True:
            activation = tf.nn.relu(activation)
        if log==True:
            tf.summary.histogram("weights",W)
            tf.summary.histogram("biases",b)
            tf.summary.histogram("activations",activation)
        return activation

In [5]:
def cnnmodel_fn(features, labels, mode, params):
    
    #### 1 INFERNCE MODEL
    
    input_layer = tf.reshape(features, [-1, 32, 32, 3])
    conv1 = _conv(input_layer,kernel=[5,5,3,128],name='conv1',log=params['log'])
    conv2 = _conv(conv1,kernel=[5,5,128,128],name='conv2',log=params['log'])
    conv3 = _conv(conv2,kernel=[3,3,128,256],name='conv3',log=params['log'])
    conv4 = _conv(conv3,kernel=[3,3,256,512],name='conv4',log=params['log'])
    dense = _dense(conv4,size_in=2*2*512,size_out=params['dense_units'],
                   name='Dense',relu=True,log=params['log'])
    if mode==tf.estimator.ModeKeys.TRAIN:
        dense = tf.nn.dropout(dense,params['drop_out'])
    logits = _dense(dense,size_in=params['dense_units'],
                    size_out=10,name='Output',relu=False,log=params['log'])
        
    #### 2 CALCULATIONS AND METRICS
    
    predictions = {"classes": tf.argmax(input=logits,axis=1),
                   "logits": logits,
                   "probabilities": tf.nn.softmax(logits,name='softmax')}
    export_outputs = {'predictions': tf.estimator.export.PredictOutput(predictions)}
    if (mode==tf.estimator.ModeKeys.TRAIN or mode==tf.estimator.ModeKeys.EVAL):
        loss = tf.losses.sparse_softmax_cross_entropy(labels=labels,logits=logits)
        #accuracy = tf.metrics.accuracy(
        #    labels=labels, predictions=tf.argmax(logits,axis=1))
        #metrics = {'accuracy':accuracy}
        
    #### 3 MODE = PREDICT
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode, predictions=predictions, export_outputs=export_outputs)

    #### 4 MODE = TRAIN

    if mode == tf.estimator.ModeKeys.TRAIN:
        learning_rate = tf.train.exponential_decay(
            params['learning_rate'],tf.train.get_global_step(),
            decay_steps=100000,decay_rate=0.96)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        if params['replicate']==True:
            optimizer = tf.contrib.estimator.TowerOptimizer(optimizer)
        train_op = optimizer.minimize(loss=loss,global_step=tf.train.get_global_step())
        tf.summary.scalar('learning_rate', learning_rate)
        #tf.summary.scalar('accuracy',accuracy[1])
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss, train_op=train_op)
    
    #### 5 MODE = EVAL
    
    if mode == tf.estimator.ModeKeys.EVAL:
        accuracy = tf.metrics.accuracy(
            labels=labels, predictions=tf.argmax(logits,axis=1))
        metrics = {'accuracy':accuracy}
        return tf.estimator.EstimatorSpec(
            mode=mode,loss=loss,eval_metric_ops=metrics)
            #mode=mode,loss=loss) #,eval_metric_ops=metrics)

In [6]:
def parse_tfrecord(example):
    feature={'idx'     : tf.FixedLenFeature((), tf.int64),
             'label'   : tf.FixedLenFeature((), tf.int64),
             'image'   : tf.FixedLenFeature((), tf.string, default_value="")}
    parsed = tf.parse_single_example(example, feature)
    image = tf.decode_raw(parsed['image'],tf.float64)
    image = tf.cast(image,tf.float32)
    image = tf.reshape(image,[32,32,3])
    return image, parsed['label']


In [7]:
def image_scaling(x):
    return tf.image.per_image_standardization(x)

def distort(x):
    x = tf.image.resize_image_with_crop_or_pad(x, 40, 40)
    x = tf.random_crop(x, [32, 32, 3])
    x = tf.image.random_flip_left_right(x)
    return x

In [8]:
def dataset_input_fn(params):
    dataset = tf.data.TFRecordDataset(
        params['filenames'],num_parallel_reads=params['threads'])
    dataset = dataset.map(parse_tfrecord, num_parallel_calls=params['threads'])
    dataset = dataset.map(lambda x,y: (image_scaling(x),y),num_parallel_calls=params['threads'])
    if params['mode']==tf.estimator.ModeKeys.TRAIN:
        dataset = dataset.map(lambda x,y: (distort(x),y),num_parallel_calls=params['threads'])
        dataset = dataset.shuffle(buffer_size=params['shuffle_buff'])
    dataset = dataset.repeat()
    dataset = dataset.batch(params['batch'])
    dataset = dataset.prefetch(2*params['batch'])
    return dataset

In [9]:
model_params  = {'drop_out'      : 0.2,
                 'dense_units'   : 1024,
                 'learning_rate' : 1e-3,
                 'log'           : True,
                 'replicate'     : False
                }

In [10]:
distribution = tf.contrib.distribute.MirroredStrategy(num_gpus=4)
config = tf.estimator.RunConfig(save_checkpoints_secs = 30,
                                keep_checkpoint_max = 5,
                                session_config=tf.ConfigProto(
                                    allow_soft_placement=True, log_device_placement=True),
                                train_distribute = distribution)
'''
config = tf.estimator.RunConfig(save_checkpoints_secs = 300,keep_checkpoint_max = 5)
'''

'\nconfig = tf.estimator.RunConfig(save_checkpoints_secs = 300,keep_checkpoint_max = 5)\n'

In [11]:
if model_params['replicate']==True:
    model_fn = tf.contrib.estimator.replicate_model_fn(
        cnnmodel_fn, loss_reduction=tf.losses.Reduction.MEAN)
else:
    model_fn = cnnmodel_fn

In [12]:
name = 'cnn_model/cnn_model_'
if model_params['replicate']==True:
    name = 'cnn_model_dist/cnn_model_'
name = name + 'dense(' + str(model_params['dense_units']) + ')_'
name = name + 'drop(' + str(model_params['drop_out']) + ')_'
name = name + 'lr(' + str(model_params['learning_rate']) + ')_'
name = name + time.strftime("%Y%m%d%H%M%S")
model_dir  = os.path.join('./',name)

print(model_dir)

./cnn_model/cnn_model_dense(1024)_drop(0.2)_lr(0.001)_20180529041424


In [13]:
model_dir='./cnn_model/cnn_model_dense(1024)_drop(0.2)_lr(0.001)_20180529041424'
model_dir

'./cnn_model/cnn_model_dense(1024)_drop(0.2)_lr(0.001)_20180529041424'

In [14]:
estimator = tf.estimator.Estimator(
    model_fn=model_fn,model_dir=model_dir,params=model_params,config=config)

INFO:tensorflow:Using config: {'_service': None, '_save_summary_steps': 100, '_task_id': 0, '_model_dir': './cnn_model/cnn_model_dense(1024)_drop(0.2)_lr(0.001)_20180529041424', '_save_checkpoints_secs': 30, '_session_config': allow_soft_placement: true
log_device_placement: true
, '_log_step_count_steps': 100, '_global_id_in_cluster': 0, '_keep_checkpoint_every_n_hours': 10000, '_tf_random_seed': None, '_master': '', '_train_distribute': <tensorflow.contrib.distribute.python.mirrored_strategy.MirroredStrategy object at 0x7fc5889efba8>, '_keep_checkpoint_max': 5, '_num_worker_replicas': 1, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc5889efe80>, '_task_type': 'worker', '_save_checkpoints_steps': None, '_evaluation_master': '', '_num_ps_replicas': 0}


In [15]:
#train_files = 'gs://tsaikevin-data/cifar10/train.tfrecords'
#val_files   = 'gs://tsaikevin-data/cifar10/validation.tfrecords'
train_files = !ls cifar10_data_00*.tfrecords
val_files   = !ls cifar10_data_01*.tfrecords

train_params = {'filenames'    : train_files,
                'mode'         : tf.estimator.ModeKeys.TRAIN,
                'threads'      : 16,
                'shuffle_buff' : 100000,
                'batch'        : 100
               }

eval_params  = {'filenames'    : val_files,
                'mode'         : tf.estimator.ModeKeys.EVAL,
                'threads'      : 8,
                'batch'        : 200
               }

train_spec = tf.estimator.TrainSpec(input_fn=lambda: dataset_input_fn(train_params),max_steps=200000)
eval_spec  = tf.estimator.EvalSpec(input_fn=lambda: dataset_input_fn(eval_params),steps=10,throttle_secs=60)


In [16]:
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 60 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Device is available but not used by distribute strategy: /device:CPU:0
INFO:tensorflow:Configured nccl all-reduce.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:batch_all_reduce invoked for batches size = 12 with algorithm = nccl and num_packs = 1
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./cnn_model/cnn_model_dense(1024)_drop(0.2)_lr(0.001

INFO:tensorflow:loss = 1.9220049, step = 3107 (4.985 sec)
INFO:tensorflow:global_step/sec: 20.9348
INFO:tensorflow:loss = 1.9471223, step = 3207 (4.776 sec)
INFO:tensorflow:global_step/sec: 36.4775
INFO:tensorflow:loss = 1.8808405, step = 3307 (2.742 sec)
INFO:tensorflow:global_step/sec: 22.9151
INFO:tensorflow:loss = 1.9213384, step = 3407 (4.364 sec)
INFO:tensorflow:global_step/sec: 22.2986
INFO:tensorflow:loss = 1.9684663, step = 3507 (4.485 sec)
INFO:tensorflow:global_step/sec: 22.6687
INFO:tensorflow:loss = 1.890625, step = 3607 (4.411 sec)
INFO:tensorflow:Saving checkpoints for 3658 into ./cnn_model/cnn_model_dense(1024)_drop(0.2)_lr(0.001)_20180529041424/model.ckpt.
INFO:tensorflow:Loss for final step: 1.8894151.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-29-04:18:04
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./cnn_model/cnn_model_dense(1024)_drop(0.2)_lr(0.001)_20180

INFO:tensorflow:batch_all_reduce invoked for batches size = 12 with algorithm = nccl and num_packs = 1
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./cnn_model/cnn_model_dense(1024)_drop(0.2)_lr(0.001)_20180529041424/model.ckpt-6160
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 6161 into ./cnn_model/cnn_model_dense(1024)_drop(0.2)_lr(0.001)_20180529041424/model.ckpt.
INFO:tensorflow:loss = 1.6867533, step = 6160
INFO:tensorflow:global_step/sec: 30.1174
INFO:tensorflow:loss = 1.6813962, step = 6260 (3.322 sec)
INFO:tensorflow:global_step/sec: 23.1836
INFO:tensorflow:loss = 1.754314, step = 6360 (4.314 sec)
INFO:tensorflow:global_step/sec: 22.6502
INFO:tensorflow:loss = 1.7543972, st

INFO:tensorflow:global_step/sec: 22.6947
INFO:tensorflow:loss = 1.5457699, step = 9662 (4.408 sec)
INFO:tensorflow:global_step/sec: 22.8277
INFO:tensorflow:loss = 1.5728667, step = 9762 (4.380 sec)
INFO:tensorflow:global_step/sec: 22.3354
INFO:tensorflow:loss = 1.6086742, step = 9862 (4.477 sec)
INFO:tensorflow:Saving checkpoints for 9913 into ./cnn_model/cnn_model_dense(1024)_drop(0.2)_lr(0.001)_20180529041424/model.ckpt.
INFO:tensorflow:Loss for final step: 1.5422335.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-29-04:23:21
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./cnn_model/cnn_model_dense(1024)_drop(0.2)_lr(0.001)_20180529041424/model.ckpt-9913
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/10]
INFO:tensorflow:Evaluation [2/10]
INFO:tensorflow:Evaluation [3/10]
INFO:tensorflow:Evaluation [4/10]
INFO:tenso

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./cnn_model/cnn_model_dense(1024)_drop(0.2)_lr(0.001)_20180529041424/model.ckpt-12415
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 12416 into ./cnn_model/cnn_model_dense(1024)_drop(0.2)_lr(0.001)_20180529041424/model.ckpt.
INFO:tensorflow:loss = 1.5092213, step = 12415
INFO:tensorflow:global_step/sec: 29.9039
INFO:tensorflow:loss = 1.4924756, step = 12515 (3.346 sec)
INFO:tensorflow:global_step/sec: 23.4418
INFO:tensorflow:loss = 1.4969249, step = 12615 (4.265 sec)
INFO:tensorflow:global_step/sec: 22.8561
INFO:tensorflow:loss = 1.5722301, step = 12715 (4.376 sec)
INFO:tensorflow:global_step/sec: 23.5011
INFO:tensorflow:loss = 1.4669495, step = 12815 (4.255 sec)
INFO:tensorflow:global_step/sec: 20.6397
INFO:tensorflow:loss = 1.4566132, step = 12915 (4.

...

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:batch_all_reduce invoked for batches size = 12 with algorithm = nccl and num_packs = 1
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./cnn_model/cnn_model_dense(1024)_drop(0.2)_lr(0.001)_20180529041424/model.ckpt-186987
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 186988 into ./cnn_model/cnn_model_dense(1024)_drop(0.2)_lr(0.001)_20180529041424/model.ckpt.
INFO:tensorflow:loss = 0.4078567, step = 186987
INFO:tensorflow:global_step/sec: 30.4524
INFO:tensorflow:loss = 0.3061696, step = 187087 (3.286 sec)
INFO:tensorflow:global_step/sec

INFO:tensorflow:global_step/sec: 20.4042
INFO:tensorflow:loss = 0.33266044, step = 190268 (4.903 sec)
INFO:tensorflow:global_step/sec: 21.1368
INFO:tensorflow:loss = 0.31879807, step = 190368 (4.729 sec)
INFO:tensorflow:global_step/sec: 36.7787
INFO:tensorflow:loss = 0.380457, step = 190468 (2.720 sec)
INFO:tensorflow:global_step/sec: 23.1946
INFO:tensorflow:loss = 0.3384465, step = 190568 (4.311 sec)
INFO:tensorflow:global_step/sec: 22.5525
INFO:tensorflow:loss = 0.32270706, step = 190668 (4.434 sec)
INFO:tensorflow:global_step/sec: 22.6759
INFO:tensorflow:loss = 0.3633591, step = 190768 (4.410 sec)
INFO:tensorflow:Saving checkpoints for 190864 into ./cnn_model/cnn_model_dense(1024)_drop(0.2)_lr(0.001)_20180529041424/model.ckpt.
INFO:tensorflow:Loss for final step: 0.370116.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-29-06:53:36
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:batch_all_reduce invoked for batches size = 12 with algorithm = nccl and num_packs = 1
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./cnn_model/cnn_model_dense(1024)_drop(0.2)_lr(0.001)_20180529041424/model.ckpt-193447
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 193448 into ./cnn_model/cnn_model_dense(1024)_drop(0.2)_lr(0.001)_20180529041424/model.ckpt.
INFO:tensorflow:loss = 0.3522219, step = 193447
INFO:tensorflow:global_step/sec: 30.826
INFO:tensorflow:loss = 0.29394275, step = 193547 (3.247 sec)
INFO:tensorflow:global_step/sec

INFO:tensorflow:loss = 0.2623356, step = 196712 (4.920 sec)
INFO:tensorflow:global_step/sec: 21.2767
INFO:tensorflow:loss = 0.38638642, step = 196812 (4.700 sec)
INFO:tensorflow:global_step/sec: 36.5624
INFO:tensorflow:loss = 0.39078346, step = 196912 (2.735 sec)
INFO:tensorflow:global_step/sec: 22.8769
INFO:tensorflow:loss = 0.2933178, step = 197012 (4.371 sec)
INFO:tensorflow:global_step/sec: 22.8181
INFO:tensorflow:loss = 0.34204873, step = 197112 (4.384 sec)
INFO:tensorflow:global_step/sec: 22.5712
INFO:tensorflow:loss = 0.2987396, step = 197212 (4.430 sec)
INFO:tensorflow:Saving checkpoints for 197305 into ./cnn_model/cnn_model_dense(1024)_drop(0.2)_lr(0.001)_20180529041424/model.ckpt.
INFO:tensorflow:Loss for final step: 0.3627947.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-29-06:58:50
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./cnn_model/cnn_model_dense(1024)_drop(0.

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:batch_all_reduce invoked for batches size = 12 with algorithm = nccl and num_packs = 1
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./cnn_model/cnn_model_dense(1024)_drop(0.2)_lr(0.001)_20180529041424/model.ckpt-199862
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 199863 into ./cnn_model/cnn_model_dense(1024)_drop(0.2)_lr(0.001)_20180529041424/model.ckpt.
INFO:tensorflow:loss = 0.31069583, step = 199862
INFO:tensorflow:global_step/sec: 30.448
INFO:tensorflow:loss = 0.36835158, step = 199962 (3.290 sec)
INFO:tensorflow:Saving checkpo